In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import input_data

In [3]:
mnist = input_data.read_data_sets('data/',one_hot=True)
trainimg = mnist.train.images
trainlabel = mnist.train.labels
testimg = mnist.test.images
testlabel = mnist.test.labels
print('MNIST READY')

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting data/t10k-images-idx3-ubyte.gz
Extracting data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
MNIST READY


In [4]:
n_input = 784  # 输入层
n_output = 10   # 输出层
# 进行参数的初始化
weights = {
    #filter的  h  w 深度  生成多少个特征图
    'wc1':tf.Variable(tf.random_normal([3,3,1,64],stddev=0.1)),
    'wc2':tf.Variable(tf.random_normal([3,3,64,128],stddev=0.1)),
    'wd1':tf.Variable(tf.random_normal([7*7*128,1024],stddev=0.1)),
    'wd2':tf.Variable(tf.random_normal([1024,n_output],stddev=0.1))
}
biases = {
    'bc1':tf.Variable(tf.random_normal([64])),
    'bc2':tf.Variable(tf.random_normal([128])),
    'db1':tf.Variable(tf.random_normal([1024])),
    'bd2':tf.Variable(tf.random_normal([n_output]))
}

In [6]:
# 卷积+池化
def conv_basic(_input,_w,_b,_keepratio):
    # input   转换为 tensorflow支持的格式   四维的 
    _input_r = tf.reshape(_input,shape=[-1,28,28,1]) #  [batch_size,h,w,深度]
    # conv layer 1   卷积
    _conv1 = tf.nn.conv2d(_input_r,filter=_w['wc1'],padding='SAME',strides=[1,1,1,1]) #strides[batchsize h w  channel ]     
    # 使用激活函数
    _conv1 = tf.nn.relu(tf.nn.bias_add(_conv1,_b['bc1']))
    # pooling1 层
    _pool1 = tf.nn.max_pool(_conv1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') # ksize batchsize h w  channels
    _pool_dr1 =  tf.nn.dropout(_pool1,_keepratio) # 输入   保留比例
    # CONV LAYER2
    _conv2 = tf.nn.conv2d(_pool_dr1,_w['wc2'],padding='SAME',strides=[1,1,1,1])
    _conv2 = tf.nn.relu(tf.nn.bias_add(_conv2,_b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    _pool_dr2 = tf.nn.dropout(_pool2,_keepratio)
    # 全连接层   先进行一个转化
    _dense1 = tf.reshape(_pool_dr2,[-1,_w['wd1'].get_shape().as_list()[0]])
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1,_w['wd1']),_b['db1']))
    _fc1_dr1 = tf.nn.dropout(_fc1,_keepratio)
    # 全连接层2
    _out = tf.add(tf.matmul(_fc1_dr1,_w['wd2']),_b['bd2'])
    
    #return 
    out = {
        'input_r':_input_r,
        'conv1':_conv1,
        'pool1':_pool1,
        'pool1_dr1':_pool_dr1,
        'conv2':_conv2,
        'pool2':_pool2,
        'pool2_dr2':_pool_dr2,
        'dense1':_dense1,
        'fc1':_fc1,
        'fc1_dr1':_fc1_dr1,
        'out':_out
    }
    return out
print('CNN READY')

CNN READY


In [30]:
a1 = tf.Variable(tf.random_normal([3,3,1,64],stddev=0.01))
print(a1)
a2 = tf.Print(a,[a],'a')
print(a2)
init1 = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init1)
    print(sess.run(a1))

<tf.Variable 'Variable_17:0' shape=(3, 3, 1, 64) dtype=float32_ref>
Tensor("Print_8:0", shape=(3, 3, 1, 64), dtype=float32)
[[[[-5.53099206e-03 -1.16641345e-02  5.09542413e-03 -5.53167542e-04
     3.19920154e-03  7.67369114e-04 -6.66990923e-03 -1.27430046e-02
     1.42608033e-02  6.97485544e-03 -9.97409504e-03 -1.80553726e-03
    -2.18676380e-03  1.09791076e-02 -9.77029838e-03 -3.05045350e-03
     3.93801648e-03  3.68185248e-03  2.37275311e-03  1.92641411e-02
     8.81556515e-03  1.25303539e-02  9.72068775e-03 -1.65132713e-02
    -5.05098980e-03  9.34018381e-03 -1.00877164e-02  2.59485114e-02
    -4.91561042e-03  6.57760119e-03 -1.76475942e-02  8.85916315e-03
    -1.08349428e-03 -1.06951632e-02 -1.54922809e-03 -6.16118824e-03
    -1.92699172e-02  5.41756512e-04 -4.49249195e-03 -1.17044784e-02
    -1.61404647e-02  4.13613860e-03  1.62409060e-02 -3.32799100e-04
     1.12967787e-03  2.38293619e-03 -1.58289969e-02 -1.86023116e-03
    -2.72689480e-03  3.74600920e-03 -1.18716538e-03 -2.32268

    -2.66479794e-02 -6.16713939e-03 -1.85691100e-03 -2.38809660e-02]]]]


In [24]:
print(help(tf.nn.max_pool))

Help on function max_pool in module tensorflow.python.ops.nn_ops:

max_pool(value, ksize, strides, padding, data_format='NHWC', name=None)
    Performs the max pooling on the input.
    
    Args:
      value: A 4-D `Tensor` of the format specified by `data_format`.
      ksize: A list or tuple of 4 ints. The size of the window for each dimension
        of the input tensor.
      strides: A list or tuple of 4 ints. The stride of the sliding window for
        each dimension of the input tensor.
      padding: A string, either `'VALID'` or `'SAME'`. The padding algorithm.
        See the "returns" section of `tf.nn.convolution` for details.
      data_format: A string. 'NHWC', 'NCHW' and 'NCHW_VECT_C' are supported.
      name: Optional name for the operation.
    
    Returns:
      A `Tensor` of format specified by `data_format`.
      The max pooled output tensor.

None


In [7]:
# 创建X Y 优化  损失
X = tf.placeholder(tf.float32,[None,n_input])
y = tf.placeholder(tf.float32,[None,n_output])
keepratio = tf.placeholder(tf.float32)

# function
_pred = conv_basic(X,_w=weights,_b=biases,_keepratio=keepratio)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=_pred,labels=y))
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
# 求准确率
_corr = tf.equal(tf.argmax(_pred,1),tf.argmax(y,1))
accr = tf.reduce_mean(tf.cast(_corr,tf.float32))

init = tf.global_variables_initializer()
print('GRAPH READY')

GRAPH READY


In [19]:
train_epochs = 30
batch_size = 16
display_step = 1
path = './log/'
with tf.Session()  as sess:
    sess.run(init)
    writer = tf.summary.FileWriter(path,sess.graph) 
    for epoch in range(train_epochs):
        avg_cost = 0
        #total_batch = int(mnist.train.num_examples/batch_size)
        total_batch = 20
        for i in range(total_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size=batch_size)
            # 开始优化
            sess.run(optm,feed_dict={X:batch_xs,y:batch_ys,keepratio:0.7})
            # 损失值
            avg_cost+=sess.run(cost,feed_dict={X:batch_xs,y:batch_ys,keepratio:1.})
        avg_cost = avg_cost/total_batch
        
        #打印
        if(epoch%display_step==0):
            print('Epoch:%03d/%03d cost: %.9f'%(epoch,train_epochs,avg_cost))
            train_acc = sess.run(accr,feed_dict={X:batch_xs,y:batch_ys,keepratio:1.})
            print('training accuracy:%.3f'%(train_acc))
#             feed_test = {X:testimg,y:testlabel,keepratio:1.}
#             test_acc = sess.run(accr,feed_dict=feed_test)
#             print('test accuracy:%.3f'%(test_acc))



Epoch:000/030 cost: 22.060002971
training accuracy:0.188
Epoch:001/030 cost: 6.416843295
training accuracy:0.062
Epoch:002/030 cost: 5.076220369
training accuracy:0.125
Epoch:003/030 cost: 5.505369473
training accuracy:0.000
Epoch:004/030 cost: 4.629777074
training accuracy:0.125
Epoch:005/030 cost: 4.127456117
training accuracy:0.312
Epoch:006/030 cost: 3.956476617
training accuracy:0.125
Epoch:007/030 cost: 3.374117720
training accuracy:0.125
Epoch:008/030 cost: 2.703554666
training accuracy:0.062
Epoch:009/030 cost: 1.928789318
training accuracy:0.188
Epoch:010/030 cost: 1.715413362
training accuracy:0.438
Epoch:011/030 cost: 1.497326666
training accuracy:0.500
Epoch:012/030 cost: 1.212003982
training accuracy:0.562
Epoch:013/030 cost: 0.838859627
training accuracy:0.625
Epoch:014/030 cost: 1.063313830
training accuracy:0.812
Epoch:015/030 cost: 0.771053877
training accuracy:0.750
Epoch:016/030 cost: 0.865377671
training accuracy:0.688
Epoch:017/030 cost: 0.681043199
training accura

In [19]:
a= np.array([[2,4],
            [4,6]])
a
with tf.Session() as sess:
    print(sess.run(tf.argmax(a,1)))
print(np.mean(a,-1))
print(a.flatten())

[1 1]
[3. 5.]
[2 4 4 6]
